# 8-1 Decision Tree
# language : Python
https://www.youtube.com/watch?v=jVh5NA9ERDA&list=PLqnslRFeH2Upcrywf-u2etjdxxkL8nl7E&index=8

## 0. 모델 기초

### Entropy
$ E = - \sum p(X) \cdot log_2(p(X))  , p(X) = \frac{\# x}{n}$

## 1. 모델 구성 